# مبدل فایل سرفصل درس 

این برنامه تبدیل فایل سرفصل درسها به صورت مارک داون که قابل درج در گیت‌هاب باشد را انجام می‌دهد.

فرض بر این هست که سرفصل همه درسها در یک فایل مارک‌داون است، این برنامه بخشهای مختلف هر درس از قبیل،‌عنوان، تعداد واحد، نوع درسها و ... را استخراج کرده و مشخصات ار درس را در یک فایل مارک‌داون قرار می‌دهد.
سپس از میک‌داکس برای ایجاد صفحات مربوطه استفاده می‌شود
ظاهرا فقط اولی جدول ابتدای هر درس رو به مارک‌داون تبدیل می‌کنه.
باقی اچ‌تی‌ام‌ال می‌کنن.

https://products.aspose.app/total/conversion

https://products.groupdocs.app/conversion/docx-to-md

## M. Amintoosi

In [269]:
# !pip install markdown-strings


In [270]:
import io
import re
import markdown_strings
import pandas as pd


In [271]:
def extract_elements(input_str):
    start_idx = input_str.find('|',1)
    start_idx = input_str.find('|',start_idx) # Move to next |
    end_idx = input_str.find('|',start_idx+1)
    prreq = input_str[start_idx+1:end_idx]
    prreq = re.sub('\*', '', prreq)
    prreq = prreq.strip()
    if prreq == '':
        prreq = 'ندارد'
    # print(prreq)

    # نوع درس
    start_idx = end_idx
    end_idx = input_str.find('|',start_idx+1)
    c_cat = input_str[start_idx+1:end_idx]
    c_cat = re.sub('\*', '', c_cat)
    if 'n' in c_cat  or '¢' in c_cat:
        c_cat = c_cat.replace('n','').strip()
        c_cat = c_cat.replace('¢','').strip()
    elif 'حل تمرین' in c_cat:
        c_cat = re.sub('حل تمرین','',c_cat)
    else:
        c_cat = ''
    # print(c_cat)

    # نوع واحد
    start_idx = end_idx
    end_idx = input_str.find('|',start_idx+1)
    unit_type = input_str[start_idx+1:end_idx]
    unit_type = re.sub('\*', '', unit_type)
    if 'n' in unit_type or '¢' in unit_type:
        unit_type = unit_type.replace('n','').strip()
        unit_type = unit_type.replace('¢','').strip()
    else:
        unit_type = ''
    # print(unit_type)
    return prreq, c_cat, unit_type

In [272]:
endinglist = ["abc", "df", "ghi"]
line = "This is a line containing def"

contains_ending = any(ending in line for ending in endinglist)
print(contains_ending)

False


In [273]:
def extract_parts(file_content, line_no, ending_list):
    part_content =[]
    line_no += 1
    line = file_content[line_no]
    while line_no<len(file_content) and not any(ending in line for ending in ending_list):
        if line.strip() != '':
            part_content.append(line)
        line_no += 1
        if line_no>= len(file_content):
            break
        line = file_content[line_no]
    return part_content, line_no

In [274]:
def extract_courses(file_path):
    # pattern = re.compile(r'عنوان درس به فارسی:(.*?)', re.UNICODE)
    with open(file_path, 'r', encoding='utf-8') as file:
        data = file.read()
        # courses = pattern.findall(data)
        courses = re.findall(r'عنوان درس به فارسی:(.*?)عنوان درس به فارسی:', data, re.DOTALL)#flags=re.UNICODE)
        # courses = re.findall(r'Stewart(.*?)Stewart', data,  re.DOTALL)#flags=re.UNICODE)
        return courses


In [275]:
# !dir

In [276]:
# استفاده از تابع برای استخراج اطلاعات درس‌ها
file_path = '../input/CS-Chart.md'
extracted_courses = extract_courses(file_path)

# نمایش نتایج
for course in extracted_courses:
    print(course)

|<h3>ریاضی عمومی ۱</h3>|
| -: | :- |
|عنوان درس به انگلیسی:|Calculus I|نوع درس و واحد|
|دروس پیش‏نیاز:|ندارد|**پایه n**|نظریn|
|دروس هم‏نیاز:|ندارد|تخصصی الزامی£|عملی £|
|تعداد واحد:|4|حل تمرین دارد|تخصصی اختیاری £|نظری-عملی £|
|تعداد ساعت:|64||رساله / پایان‏نامه £||
اهداف درس: آشنایی دانشجویان با مفاهیم اصلی حساب دیفرانسیل و انتگرال یک متغیره و کاربردهای آنها 

توانایی و شایستگی‌هایی که درس پرورش می‌دهد: توانمندی دانشجویان در مباحث نظری ریاضی و به کارگیری آن در دروس دیگر

**سرفصل درس:** 

مختصات دکارتی، مختصات قطبی، اعداد مختلط و جمع و ضرب و ریشه آنها، نمایش های مختلف اعداد مختلط، دنباله های عددی، حد و قضایای مربوط، حد بینهایت و حد در بینهایت، حد چپ و راست،  پیوستگی، پیوستگی دنباله ای، قضیه مقدار میانی برای توابع پیوسته، قضیه مقدار اکسترمم برای توابع پیوسته، مشتق، دستورهای مشتق گیری، تابع معکوس و مشتق آن، مشتق توابع مثلثاتی و توابع معکوس آنها، قضیه مشتق ترکیب توابع، قضیه فرما (صفر شدن مشتق در یک نقطه اکسترمم)، قضیه رل، قضیه مقدار میانگین، قضیه مقدار میانگین کشی، آزمون مشتق اول و دوم ب

In [277]:
len(extracted_courses), extracted_courses

(40,
 ['|<h3>ریاضی عمومی ۱</h3>|\n| -: | :- |\n|عنوان درس به انگلیسی:|Calculus I|نوع درس و واحد|\n|دروس پیش\u200fنیاز:|ندارد|**پایه n**|نظریn|\n|دروس هم\u200fنیاز:|ندارد|تخصصی الزامی£|عملی £|\n|تعداد واحد:|4|حل تمرین دارد|تخصصی اختیاری £|نظری-عملی £|\n|تعداد ساعت:|64||رساله / پایان\u200fنامه £||\nاهداف درس: آشنایی دانشجویان با مفاهیم اصلی حساب دیفرانسیل و انتگرال یک متغیره و کاربردهای آنها \n\nتوانایی و شایستگی\u200cهایی که درس پرورش می\u200cدهد: توانمندی دانشجویان در مباحث نظری ریاضی و به کارگیری آن در دروس دیگر\n\n**سرفصل درس:** \n\nمختصات دکارتی، مختصات قطبی، اعداد مختلط و جمع و ضرب و ریشه آنها، نمایش های مختلف اعداد مختلط، دنباله های عددی، حد و قضایای مربوط، حد بینهایت و حد در بینهایت، حد چپ و راست،  پیوستگی، پیوستگی دنباله ای، قضیه مقدار میانی برای توابع پیوسته، قضیه مقدار اکسترمم برای توابع پیوسته، مشتق، دستورهای مشتق گیری، تابع معکوس و مشتق آن، مشتق توابع مثلثاتی و توابع معکوس آنها، قضیه مشتق ترکیب توابع، قضیه فرما (صفر شدن مشتق در یک نقطه اکسترمم)، قضیه رل، قضیه مقدار میانگین، 

In [278]:
# Load the file into file_content
file_content = [line for line in open('../input/CS-Chart.md')]
file_content = [line.replace('**منابع**:', '**منابع:**') for line in file_content]
col_names = ['fa_title','en_title','en_file_name','c_cat','complementary','prreq','coreq',\
                           'c_unit','unit_type','c_hours','c_ex','c_goal1','c_goal2','sar_fasl','c_refs']
df = pd.DataFrame(columns=col_names)

fa_col_names = ['عنوان درس به فارسی:', 'عنوان درس به انگلیسی:', 'نوع درس:', 'آموزش تکمیلی:','پیش‌نیاز:','هم‌نیاز',\
                 'تعداد واحد:', 'نوع واحد:', 'تعداد ساعت:','حل تمرین:', 'اهداف درس:', 'توانایی و شایستگی‌هایی که درس پرورش می‌دهد:',\
                      'سرفصل درس:','منابع:']
# df.set_index('fa_title',inplace = True)
line_no = 0
# line = file_content[line_no]
# while line_no < len(file_content) and line.find(fa_col_names[0]) == -1:
#     line_no += 1
#     line = file_content[line_no]
# # line_no -=1

while line_no < len(file_content):
    line = file_content[line_no]
    pattern = r"\| --- \|"
    result = re.sub(pattern, "", line)
    # pattern = r"\|"
    # result = re.sub(pattern, "", result)
    srch_str = "عنوان درس به فارسی:"
    # print(result, result.find(srch_str))
    if result.find(srch_str) != -1:
        result = re.sub("<a.*?/a>",'',result)
        start_idx = result.find('<h3>') + 4
        end_idx = result.rfind('</h3>')
        fa_title = result[start_idx:end_idx]
        fa_title = re.sub('\*', '', fa_title)
        fa_title = fa_title.strip()
        line_no += 1
        print(fa_title)
        continue

    # line_no += 2 # for | -: | :- |

    srch_str = "عنوان درس به انگلیسی:"
    # print(result , result.find(srch_str))
    if result.find(srch_str) != -1:
        start_idx = result.find('|',1)
        end_idx = result.find('|',start_idx+1)
        en_title = result[start_idx+1:end_idx]
        en_title = re.sub('\*', '', en_title)
        en_title = re.sub('<.*?>', '', en_title)
        en_title = en_title.strip()
        en_file_name = re.sub('\&', 'and', en_title)
        en_file_name = re.sub('\n', '', en_file_name)
        en_file_name = re.sub(' ', '-', en_file_name)
        # print(en_title)

        # پیش نیاز
        line_no += 1
        line = file_content[line_no]
        prreq, c_cat, unit_type = extract_elements(line)
        # print(prreq, c_cat, unit_type)

        line_no += 1
        line = file_content[line_no]
        coreq, c_cat_, unit_type_ = extract_elements(line)
        # if c_cat_ != '':
        #     print('\n', c_cat_, '\n')
        c_cat, unit_type = c_cat+c_cat_,  unit_type+unit_type_
        # print(coreq, c_cat, unit_type)

        line_no += 1
        line = file_content[line_no]
        c_unit, c_ex, c_cat_ = extract_elements(line)
        if unit_type == '':
            unit_type = "نظری-عملی"
        c_cat = c_cat+c_cat_
        # print(c_unit, c_ex, c_cat)

        line_no += 1
        line = file_content[line_no]
        c_hours, _, _ = extract_elements(line)
        print(c_hours, '\n')
       
        # common_ending_list = ['برنامه‌ی هشت ترمی','دروس تخصصی', 'دروس الزامی']

        while True:
            line_no += 1
            if line_no >= len(file_content):
                break
            line = file_content[line_no]
            if line.find(fa_col_names[-2]) != -1: #سرفصل
                break
        if line_no >= len(file_content):
                break

        # continue
        # خواندن سرفصل
        
        # ending_list = common_ending_list.copy()
        # ending_list.append(fa_col_names[-1])
        # sar_fasl, line_no = extract_parts(file_content, line_no, ending_list)

        sar_fasl =[]
        line_no += 1
        line = file_content[line_no]
        while line_no<len(file_content) and line.find(fa_col_names[-1]) == -1: #refs
            if line.strip() != '':
                sar_fasl.append(line)
            line_no += 1
            if line_no>= len(file_content):
                break
            line = file_content[line_no]

        if line_no >= len(file_content):
            break    

        # ending_list = common_ending_list.copy()
        # ending_list.append('عنوان درس به فارسی')
        # refs, line_no = extract_parts(file_content, line_no, ending_list)
        refs = []
        line_no += 1
        line = file_content[line_no]
        while line_no<len(file_content) and line.find("عنوان درس به فارسی:") == -1:
            if line.strip() != '':
                refs.append(line)
            line_no += 1
            if line_no>= len(file_content):
                break
            line = file_content[line_no]
        #     # print(line_no)

        if line.find("عنوان درس به فارسی:") != -1:
            line_no -= 1
        this_course ={'fa_title':fa_title, 'en_title':en_title, 'en_file_name':en_file_name,'c_cat':c_cat,'prreq':prreq,'coreq':coreq,\
                           'c_unit':c_unit,'unit_type':unit_type,'c_hours':c_hours,'c_ex':c_ex,\
                            'sar_fasl':sar_fasl,'c_refs':refs}
        df = df.append(this_course,ignore_index=True)
    line_no += 1
    # print('')
    
df.head(7)

ریاضی عمومی ۱
64 

ریاضی عمومی ۲
64 

معادلات دیفرانسیل
48 

مبانی کامپیوتر و برنامه‌سازی
48 

مبانی علوم ریاضی
48 

آمار و احتمال ۱
48 

نرم‌افزارهای آماری و تحلیل داده‌ها
32 

کارگاه کامپیوتر ۱
32 

برنامه‌سازی پیشرفته
48 

برنامه‌نویسی پایتون
48 

ساختمان داده‌ها
48 

طراحی و تحلیل الگوریتم‌ها
48 

کارگاه کامپیوتر ۲
32 

پایگاه داده‌ها
48 

اصول سیستم‌های کامپیوتری
48 

مبانی ترکیبیات
64 

مبانی منطق و نظریه مجموعه‌ها
48 

مبانی نظریه محاسبه
48 

مبانی محاسبات علمی
48 

مبانی آنالیز عددی
48 

مبانی آنالیز ریاضی
48 

تحلیل آماری داده‌ها
48 

سری‌های زمانی کاربردی
48 

یادگیری ماشین مقدماتی
48 

داده‌کاوی مقدماتی
48 

تحقیق در عملیات
48 

بهینه‌سازی غیرخطی
48 

طراحی و توسعه کسب و کارهای نوپا
48 

مبانی اقتصاد دیجیتال
32 

کارآموزی
192 

هوش مصنوعی
48 

آشنایی با یادگیری عمیق
48 

بهینه‌سازی گسسته
48 

آمار و احتمال ۲
48 

آمار محاسباتی
48 

روش‌های آماری
48 

نظریه محاسبه
48 

منطق برای علوم کامپیوتر
48 

نظریه مقدماتی کدگذاری
48 

رمزنگاری
48 

برنامه‌نویسی امن
48 

معناشناسی عملیات

 

کامپایلر
48 

مباحثی در علوم کامپیوتر ۱
48 

مباحثی در علوم کامپیوتر ۲
48 

مبانی بیوانفورماتیک
48 

پروژه کارشناسی
96 

آنالیز عددی
48 

آشنایی با کلان داده‌ها
48 

فرآیندهای تصادفی
48 

مبانی رایانش ابری
48 

رایانش چند‌هسته‌ای
48 

سیگنال‌ها و سیستم‌ها
48 

مدیریت پروژه‌های فناوری اطلاعات
48 

جبر خطی
48 

بازی‌سازی و بازی‌انگاری
48 

زبان‌های برنامه‌سازی
48 

آشنایی با پردازش زبان طبیعی
48 



,fa_title,en_title,en_file_name,c_cat,complementary,prreq,coreq,c_unit,unit_type,c_hours,c_ex,c_goal1,c_goal2,sar_fasl,c_refs
0,ریاضی عمومی ۱,Calculus I,Calculus-I,پایه,NaN,ندارد,ندارد,4,نظری,64,دارد,NaN,NaN,[مختصات دکارتی، مختصات قطبی، اعداد مختلط و جمع...,"[-\n, - - استوارت، ج. (1395). حساب دیفرانسیل و..."
1,ریاضی عمومی ۲,Calculus II,Calculus-II,پایه,NaN,ریاضی عمومی ۱,ندارد,4,نظری,64,دارد,NaN,NaN,[فضای اقلیدسی، معادلات پارامتری، مختصات فضائی،...,"[-\n, - - استوارت، ج. (1395). حساب دیفرانسیل و..."
2,معادلات دیفرانسیل,Differential Equations,Differential-Equations,پایه,NaN,ریاضی عمومی ۱,ندارد,3,نظری,48,دارد,NaN,NaN,[مفاهیم و تعاریف اولیه .معادلات جداپذیر و معاد...,"[-\n, - - ظهوری زنگنه، ح. (مترجم). (1394). معا..."
3,مبانی کامپیوتر و برنامه‌سازی,Fundamentals of Computer Programming,Fundamentals-of-Computer-Programming,پایه,NaN,ندارد,ندارد,3,نظری,48,دارد,NaN,NaN,[اهداف درس با هر زبان برنامه‌نویسی سطح بالا مح...,"[-\n, - - Matthes, E. (2019). Python Crash Cou..."
4,مبانی علوم ریاضی,Foundation of Mathematics,Foundation-of-Mathematics,پایه,NaN,ندارد,ندارد,3,نظری,48,دارد,NaN,NaN,[منطق گزاره ها و جدول ارزش. مجموعه‌ها و اعمال ...,"[-\n, - - موحد، ضیاء (1379). درآمدی به منطق جد..."
5,آمار و احتمال ۱,Probability and Statistics I,Probability-and-Statistics-I,پایه,NaN,ندارد,ندارد,3,نظری,48,دارد,NaN,NaN,[احتمال: فضای احتمال، قوانین احتمال، مدل های ا...,"[-\n, - - بهبودیان، جواد (1394). آمار و احتمال..."
6,نرم‌افزارهای آماری و تحلیل داده‌ها,Statistical Software & data analysis,Statistical-Software-and-data-analysis,تخصصی الزامی,NaN,ندارد,ندارد,1,عملی,32,ندارد,NaN,NaN,[تعریف علم آمار، بیان کاربردهای آمار در سایر ع...,"[-\n, - - گوس، جی و منیروپ، د. (1401). آمار با..."


In [279]:
# df.head(7)

In [280]:
# # می‌خواستم کاراکتر خط جدید از جدول روش یاددهی رو حذف کنم. چون در خروجی یک خط فاصله می‌افتاد و جدول نشون داده نمیشه  
# # نام ستونی که روی عناصر آن حرکت می‌کنیم
# column_name = "sar_fasl"

# # حلقه بر روی عناصر ستون خاص
# for i in range(len(df)):
#     sar_fasl = df.loc[i, column_name]
#     print(sar_fasl)
#     # حذف از بخش روش یاددهی به بعد
#     index = None
#     new_sar_fasl = []
#     for j, item in enumerate(sar_fasl):
#         if item.startswith('|'):
#             new_sar_fasl.append(item.strip())
#         else:
#             new_sar_fasl.append(item)
#     print(new_sar_fasl)
#     df.at[i, column_name] = new_sar_fasl


In [281]:
# df
# df['sar_fasl'][10]

In [282]:
# می‌خواستم از بخش روش یاددهی به بعد که در مراحل قبل به عنوان سرفصل خوانده شده رو حذف کنم
# نام ستونی که روی عناصر آن حرکت می‌کنیم
column_name = "sar_fasl"

# حلقه بر روی عناصر ستون خاص
for i in range(len(df)):
    sar_fasl = df.loc[i, column_name]
    print(sar_fasl)
    # حذف از بخش روش یاددهی به بعد
    index = None
    for j, item in enumerate(sar_fasl):
        if "روش یاددهی-یادگیری" in item:
            index = j
            break

    # حذف عناصر از ایندکس  به بعد
    if index is not None:
        sar_fasl = sar_fasl[:index]
        print('After prune:', type(sar_fasl))
        df.at[i, column_name] = sar_fasl
# A = df['sar_fasl'][20]

df['sar_fasl'][10]

['مختصات دکارتی، مختصات قطبی، اعداد مختلط و جمع و ضرب و ریشه آنها، نمایش های مختلف اعداد مختلط، دنباله های عددی، حد و قضایای مربوط، حد بینهایت و حد در بینهایت، حد چپ و راست،  پیوستگی، پیوستگی دنباله ای، قضیه مقدار میانی برای توابع پیوسته، قضیه مقدار اکسترمم برای توابع پیوسته، مشتق، دستورهای مشتق گیری، تابع معکوس و مشتق آن، مشتق توابع مثلثاتی و توابع معکوس آنها، قضیه مشتق ترکیب توابع، قضیه فرما (صفر شدن مشتق در یک نقطه اکسترمم)، قضیه رل، قضیه مقدار میانگین، قضیه مقدار میانگین کشی، آزمون مشتق اول و دوم برای اکسترمم ها، تقعر منحنی، نقطه عطف، دیفرانسیل یک تابع، بسط تیلر، کاربردهای هندسی و فیزیکی مشتق، خم ها، سرعت و شتاب در مختصات قطبی، کاربرد مشتق در تقریب ریشه های معادلات، تعریف انتگرال توابع و انتگرال پذیری، قضیه مقدار میانگین برای انتگرال ها، قضایای اساسی حساب دیفرانسیل و انتگرال، تابع اولیه، روش های تقریبی برآورد انتگرال، کاربرد انتگرال در محاسبه مساحت و حجم و طول منحنی و ...، لگاریتم و تابع نمایی و مشتق آنها، تابع های هذلولوی، روش های انتگرال گیری (همه روش ها)، آشنایی با تابع گاما و خ

['آنالیز پیچیدگی زمان و حافظه الگوریتم\u200cها، توابع بازگشتی، معرفی ساختمان\u200cهای داده\u200cای مقدماتی (لیست پیوندی، پشته، صف و صف اولویت\u200cدار) و الگوریتم\u200cهای وابسته به آن\u200cها،  نمایش ماتریس\u200cها در حافظه، ماتریس\u200cهای تنک،\u200c تطابق رشته، درخت\u200cها و الگوریتم\u200cهای پیمایش مربوطه، درخت عبارت، تبدیل نگارش\u200cهای مختلف یک عبارت ریاضی، درهم\u200cسازی، گراف\u200cها و الگوریتم\u200cهای مربوط به گراف (BFS، DFS و ...) ، الگوریتم\u200cهای جستجو و مرتب\u200cسازی.\n']

In [283]:
df.to_excel('output/lessons.xlsx')

In [284]:
def table_from_rows(table_list):
    """Return a formatted table, using each list as the list. The specifics are
    the same as those for the table function.

    >>> table_from_rows([["1","2","3"],["4","5","6"],["7","8","9"]])
    '| 1 | 2 | 3 |\\n| --- | --- | --- |\\n| 4 | 5 | 6 |\\n| 7 | 8 | 9 |'
    """
    # transpose the list
    number_of_rows = len(table_list[0])
    transposed = []
    for column_number in range(0, number_of_rows):
        column_list = [row[column_number] for row in table_list]
        transposed.append(column_list)

    return markdown_strings.table(transposed)


شناسایی دسته هر درس، مشخص کردن نام فایل درس

In [285]:
for index, row in df.iterrows():
    # print(row['fa_title'], row['en_title'])
    if 'پایه' in  row['c_cat']:
        r_path = 'base'
    elif 'الزامی' in row['c_cat']:
        r_path = 'mandatory'
    elif 'اختیاری' in row['c_cat']:
        r_path = 'elective'
    # print(r_path)
    lesson_name_en = row['en_title'].strip()
    lesson_name_en = re.sub('\&', 'and', lesson_name_en)
    lesson_name_en = re.sub(' ', '-', lesson_name_en)
    lesson_file_name = r_path+'/'+lesson_name_en+'.md'
    row['en_file_name'] = lesson_file_name
    
# df.head()   

In [286]:
df

,fa_title,en_title,en_file_name,c_cat,complementary,prreq,coreq,c_unit,unit_type,c_hours,c_ex,c_goal1,c_goal2,sar_fasl,c_refs
0,ریاضی عمومی ۱,Calculus I,base/Calculus-I.md,پایه,NaN,ندارد,ندارد,4,نظری,64,دارد,NaN,NaN,[مختصات دکارتی، مختصات قطبی، اعداد مختلط و جمع...,"[-\n, - - استوارت، ج. (1395). حساب دیفرانسیل و..."
1,ریاضی عمومی ۲,Calculus II,base/Calculus-II.md,پایه,NaN,ریاضی عمومی ۱,ندارد,4,نظری,64,دارد,NaN,NaN,[فضای اقلیدسی، معادلات پارامتری، مختصات فضائی،...,"[-\n, - - استوارت، ج. (1395). حساب دیفرانسیل و..."
2,معادلات دیفرانسیل,Differential Equations,base/Differential-Equations.md,پایه,NaN,ریاضی عمومی ۱,ندارد,3,نظری,48,دارد,NaN,NaN,[مفاهیم و تعاریف اولیه .معادلات جداپذیر و معاد...,"[-\n, - - ظهوری زنگنه، ح. (مترجم). (1394). معا..."
3,مبانی کامپیوتر و برنامه‌سازی,Fundamentals of Computer Programming,base/Fundamentals-of-Computer-Programming.md,پایه,NaN,ندارد,ندارد,3,نظری,48,دارد,NaN,NaN,[اهداف درس با هر زبان برنامه‌نویسی سطح بالا مح...,"[-\n, - - Matthes, E. (2019). Python Crash Cou..."
4,مبانی علوم ریاضی,Foundation of Mathematics,base/Foundation-of-Mathematics.md,پایه,NaN,ندارد,ندارد,3,نظری,48,دارد,NaN,NaN,[منطق گزاره ها و جدول ارزش. مجموعه‌ها و اعمال ...,"[-\n, - - موحد، ضیاء (1379). درآمدی به منطق جد..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,مدیریت پروژه‌های فناوری اطلاعات,Information Technology Project Management,elective/Information-Technology-Project-Manage...,تخصصی اختیاری,NaN,ندارد,ندارد,3,نظری,48,ندارد,NaN,NaN,[مقدمات، اهداف، برنامه‌ی درسی و توافق بر الگوی...,"[-\n, - - Marchewka, J. T. (2014). Information..."
76,جبر خطی,Linear Algebra,elective/Linear-Algebra.md,تخصصی اختیاری,NaN,مبانی محاسبات علمی,ندارد,3,نظری,48,دارد,NaN,NaN,[مروری بربردارها و ماتریسها:جمع وتفریق و ضرب م...,[- - داتا، ب.، (۱۳۹۵). جبر خطی عددی و کاربردها...
77,بازی‌سازی و بازی‌انگاری,Gamification and Game Design,elective/Gamification-and-Game-Design.md,تخصصی اختیاری,NaN,آمار و احتمال ۱، مبانی ترکیبیات، برنامه‌سازی پ...,ندارد,3,نظری,48,دارد,NaN,NaN,[بازی چیست؟، انواع و دسته‌بندی بازیها، چالش و...,"[-\n, - - Schell, J. (2008). The Art of Game D..."
78,زبان‌های برنامه‌سازی,Programming Languages,elective/Programming-Languages.md,تخصصی اختیاری,NaN,ساختمان داده‌ها,ندارد,3,نظری,48,,NaN,NaN,[تاریخچه و انواع زبانهای برنامه نویسی (رویه‌ای...,"[-\n, - - Sebesta, R. W. (2012). Concepts of p..."


In [287]:
def prreq_links(prreq_str, path_mod = ''):
# تابعی برای استخراج نام هر درس از لیست درسهای پیش‌نیازی و درج لینک فایل مربوطه
    # prreq_str = "مبانی آنالیز ریاضی، مبانی ماتریس‌ها و جبر خطی "
    p_strs = prreq_str.split('،')
    p_list = []
    for p in p_strs:
        x = df[df['fa_title'].str.contains(p.strip())][['fa_title','en_file_name']]
        p_str = p
        if not x.empty:
            fa_title, en_file_name = x['fa_title'].values[0], x['en_file_name'].values[0]
            p_str = '['+fa_title+']('+path_mod+en_file_name+')'
        p_list.append(p_str)
    return ' ،'.join(p_list)


In [288]:
# prreq_str = "مبانی آنالیز ریاضی، مبانی ماتریس‌ها و جبر خطی ، مبانی آنالیز ریاضی، ریاضی عمومی ۱"
# print(prreq_links(prreq_str))


ایجاد فایل برای تک تک درسها با فرمت راست به چپ

In [289]:
# for index, row in df.iterrows():
#     # # print(row['fa_title'], row['en_title'])
#     # if 'پایه' in  row['c_cat']:
#     #     r_path = 'base'
#     # elif 'الزامی' in row['c_cat']:
#     #     r_path = 'mandatory'
#     # elif 'اختیاری' in row['c_cat']:
#     #     r_path = 'elective'
#     # # print(r_path)
#     # lesson_name_en = row['en_title'].strip()
#     # lesson_name_en = re.sub('\&', 'and', lesson_name_en)
#     # lesson_name_en = re.sub(' ', '-', lesson_name_en)
#     # lesson_file_name = r_path+'/'+lesson_name_en+'.md'
#     lesson_file_name = row['en_file_name']
#     lesson_writer = open(lesson_file_name, 'w')
#     md_str = markdown_strings.header(row['fa_title']+'\n', 1)
#     lesson_writer.write(md_str)
#     md_str = markdown_strings.header(row['en_title']+'\n', 2)
#     lesson_writer.write(md_str)
#     lesson_writer.write(markdown_strings.horizontal_rule())
#     lesson_writer.write('\n')
#     lesson_writer.write(table_from_rows([\
#         ["نام درس:",row['fa_title'],"مقطع:", "کارشناسی"],\
#         ["پیش‌نیاز:",prreq_links(row['prreq'],path_mod = '../'),"گروه درس:",row['c_cat']],\
#         ["هم‌نیاز:",row['coreq'],"نوع درس:",row['unit_type']],\
#         ["تعداد واحد:",row['c_unit'],"تعداد ساعت:",row['c_hours']],\
#         ["حل تمرین:",row['c_ex'],"",""]\
#         ]))
#     lesson_writer.write('\n\n')
#     lesson_writer.write('**سر فصل یا رئوس مطالب**\n\n')
#     lesson_writer.write("\n".join(row['sar_fasl']))
#     # for line in row['sar_fasl']:
#     #     lesson_writer.write(f"{line}\n")
#     lesson_writer.write('\n**فهرست منابع پیشنهادی**\n\n')
#     lesson_writer.write("\n".join(row['c_refs']))
#     lesson_writer.close()
#     # if index > 15:
#     #     break

حذف فایلهای پوشه‌ها

In [290]:
import os
def clear_doc_folders(docs_path):
    for path in ['base', 'elective', 'mandatory']:
        directory = os.path.join(docs_path,path)
        # تمام فایل‌ها و زیرمجموعه‌های آن را بدست آورید
        for root, dirs, files in os.walk(directory):
            for file in files:
                file_path = os.path.join(root, file)
                # بررسی حجم فایل و نام فایل
                # os.path.getsize(file_path) != 0 and
                if  file != 'README.md':
                    # حذف فایل
                    os.remove(file_path)

In [291]:
clear_doc_folders('c:/temp/git/mkdocs-base/docs/')

ایجاد فایل برای تک تک درسها با فرمت چپ به راست

In [292]:
# ref_file_name = 'output/refs.txt'
# ref_writer = open(ref_file_name, 'w')

for index, row in df.iterrows():
    lesson_file_name = row['en_file_name']
    lesson_writer = open(lesson_file_name, 'w')
    md_str = markdown_strings.header(row['fa_title']+'\n', 1)
    lesson_writer.write(md_str)
    md_str = markdown_strings.header(row['en_title']+'\n', 2)
    lesson_writer.write(md_str)
    lesson_writer.write(markdown_strings.horizontal_rule())
    lesson_writer.write('\n')
    lesson_writer.write(table_from_rows([\
        ["نام درس:",row['fa_title'],"مقطع:", "کارشناسی"][::-1],\
        ["پیش‌نیاز:",prreq_links(row['prreq'],path_mod = '../'),"گروه درس:",row['c_cat']][::-1],\
        ["هم‌نیاز:",row['coreq'],"نوع درس:",row['unit_type']][::-1],\
        ["تعداد واحد:",row['c_unit'],"تعداد ساعت:",row['c_hours']][::-1],\
        ["حل تمرین:",row['c_ex'],"",""][::-1]\
        ]))
    lesson_writer.write('\n\n')
    lesson_writer.write('**سرفصل درس:**\n\n')
    tmp_lines = [re.sub('-\n', '', rf) for rf in row['sar_fasl']]
    tmp_lines = [re.sub('- -', '-', rf) for rf in tmp_lines]
    lesson_writer.write("\n".join(tmp_lines))
    # lesson_writer.write("\n".join(row['sar_fasl']))
    # for line in row['sar_fasl']:
    #     lesson_writer.write(f"{line}\n")
    lesson_writer.write('\n**منابع:**\n\n')
    tmp_lines = [re.sub('-\n', '', rf) for rf in row['c_refs']]
    tmp_lines = [re.sub('- -', '-', rf) for rf in tmp_lines]
    lesson_writer.write("\n".join(tmp_lines))
    lesson_writer.close()

#     ref_writer.write("\n\n"+index.__str__()+" - "+ row['fa_title']+"\n")
#     refs = [re.sub('\*', '', rf) for rf in row['c_refs']]
#     # print(refs)
#     ref_writer.write("".join(refs))

# ref_writer.close()


In [293]:
# print(["تعداد واحد:",row['c_unit'],"تعداد ساعت:",row['c_hours']].reverse())
row_el = ["نام درس:",row['fa_title'],"مقطع:", "کارشناسی"]
print(list(row_el))
["نام درس:",row['fa_title'],"مقطع:", "کارشناسی"][::-1]
# row['fa_title']

['نام درس:', 'آشنایی با پردازش زبان طبیعی', 'مقطع:', 'کارشناسی']


['کارشناسی', 'مقطع:', 'آشنایی با پردازش زبان طبیعی', 'نام درس:']

ایجاد فایل yml

In [294]:
import locale
locale.setlocale(locale.LC_ALL, 'fa_IR.utf8')
# نام درسها رو بر اساس ترتیب صحیح الفبای فارسی مرتب می‌کنیم
# اما نام دسته را بر همان اساس ترتیب نادرست که پ آخره و در ترتیب معکوس، میشه اول
df_temp = df.sort_values(by = 'fa_title', key = lambda col: col.map(lambda x: locale.strxfrm(x)))
df_sorted = df_temp.sort_values(by = 'c_cat', ascending=[False], kind = 'mergesort')
# df_sorted = df.sort_values(by=['c_cat', 'fa_title'], ascending=[False,True])
# df_sorted

In [295]:
# df['c_cat'].head()
# df_sorted
# locale.strxfrm('پایه') < locale.strxfrm('ان')

In [296]:
yml_file_name = '../mkdocs.yml'
yml_writer = open(yml_file_name, 'w')
yml_str = "site_name: علوم کامپیوتر فردوسی \n\
nav:\n\
  - صفحه اصلی: index.md\n\
  - دروس پایه:\n\
"
yml_writer.write(yml_str)
cat_changed = False
r_path = 'base'
prev_cat = r_path
for index, row in df_sorted.iterrows():
    if 'پایه' in  row['c_cat']:
        r_path = 'base'
        curr_cat = r_path
    elif 'الزامی' in row['c_cat']:
        r_path = 'mandatory'
        curr_cat = r_path
        yml_str = "  - دروس الزامی:"
    elif 'اختیاری' in row['c_cat']:
        r_path = 'elective'
        curr_cat = r_path
        yml_str = "  - دروس اختیاری:"

    if curr_cat != prev_cat:
        yml_writer.write(yml_str)
        yml_writer.write('\n')
    prev_cat = curr_cat
    # print(r_path)
    # lesson_file_name = r_path+'/'+row['en_file_name']+'.md'
    # print(lesson_file_name)
    # yml_str = '    - "'+row['fa_title']+'": '+lesson_file_name+'\n'
    yml_str = '    - "'+row['fa_title']+'": '+row['en_file_name']+'\n'
    yml_writer.write(yml_str)

    # if index > 80:
    #     break

# yml_writer.write('\n\ntheme: jinks_en')
yml_writer.write('\n\ntheme:\n')
yml_writer.write('    name: mkdocs\n')
yml_writer.write('    locale: fa\n')
yml_writer.close()


تبدیل سرفصلها با سایت زیر
https://products.aspose.app/words/conversion/word-to-md

تبدیل ایندکس با سایت زیر
https://alldocs.app/convert-word-docx-to-markdown


In [297]:
# !pip install mkdocs-jinks

!mkdocs serve
خطا داشت!

And then: mkdocs build

Copy site folder contents to cs-um/docs

In [298]:
# !mkdocs build

## Generate README.md, SUMMARY.md for gitbook

in https://cs-um.gitbook.io/curr/

In [299]:
rdme_file_name = '../README.md'
rdme_writer = open(rdme_file_name, 'w')
rdme_str = '# خوش آمدید \n\
{% hint style="info" %}\n\
**درباره این سایت:** این سایت صفحه غیررسمی گروه علوم کامپیوتر و مرتبط با برنامه درسی دوره کارشناسی علوم کامپیوتر فردوسی می‌باشد\n\
سایت، غیر رسمی و در حال راه‌اندازی است و مطالب آن قابلیت استناد ندارند.\n\
{% endhint %}\n\n\
<details>\n\
<summary>دروس پایه</summary>\n\n\
'
rdme_writer.write(rdme_str)
cat_changed = False
r_path = 'base'
prev_cat = r_path
for index, row in df_sorted.iterrows():
    if 'پایه' in  row['c_cat']:
        r_path = 'base'
        curr_cat = r_path
    elif 'الزامی' in row['c_cat']:
        r_path = 'mandatory'
        curr_cat = r_path
        rdme_str = "دروس الزامی"
    elif 'اختیاری' in row['c_cat']:
        r_path = 'elective'
        curr_cat = r_path
        rdme_str = "دروس اختیاری"

    if curr_cat != prev_cat:
        rdme_writer.write('\n</details>\n')
        rdme_writer.write('\n<details>\n')
        rdme_writer.write('\n<summary>')
        rdme_writer.write(rdme_str)
        rdme_writer.write('</summary>\n')
        rdme_writer.write('\n')
    prev_cat = curr_cat
    # print(r_path)
    # lesson_file_name = 'https://cs-um.gitbook.io/curr/docs/'+r_path+'/'+row['en_file_name']
    # lesson_file_name = 'docs/'+r_path+'/'+row['en_file_name']+'.md'
    lesson_file_name = 'docs/'+r_path+'/'+row['en_file_name']
    # print(lesson_file_name)
    rdme_str = '* ['+row['fa_title']+']('+row['en_file_name']+')\n'
    rdme_writer.write(rdme_str)

    # if index > 80:
    #     break

rdme_writer.write('\n</details>\n')
rdme_writer.close()


In [300]:
# For Gitbook
rdme_file_name = '../Tables.md'
rdme_writer = open(rdme_file_name, 'w')
rdme_str = '\
---\n\
dir: rtl\n\
---\n\n\
# جداول دروس پایه، تخصصی و اختیاری\n\n'
rdme_writer.write(rdme_str)
rdme_writer.write('\n## '+'دروس پایه\n\n')
rdme_writer.write(table_from_rows([["تعداد واحد","پیش‌نیاز","نام درس"]]))
num_units = 0
cat_changed = False
r_path = 'base'
prev_cat = r_path
for index, row in df_sorted.iterrows():
    if 'پایه' in  row['c_cat']:
        r_path = 'base'
        curr_cat = r_path
    elif 'الزامی' in row['c_cat']:
        r_path = 'mandatory'
        curr_cat = r_path
        rdme_str = "دروس الزامی"
    elif 'اختیاری' in row['c_cat']:
        r_path = 'elective'
        curr_cat = r_path
        rdme_str = "دروس اختیاری"
    if curr_cat != prev_cat:
        rdme_writer.write('\n |'+ num_units.__str__()+'|'+'مجموع تعداد واحد'+'|\n')
        num_units = 0
        rdme_writer.write('\n## '+rdme_str)
        rdme_writer.write('\n\n')
        rdme_writer.write(table_from_rows([["تعداد واحد","پیش‌نیاز","نام درس"]]))

    prev_cat = curr_cat
    num_units += int(row['c_unit'])
    rdme_writer.write('\n|'+row['c_unit']+'|'+\
        prreq_links(row['prreq'],path_mod = 'docs/') +'|'+\
            '['+row['fa_title']+'](docs/'+row['en_file_name']+') | ')
    # if index > 80:
    #     break
rdme_writer.write('\n |'+ num_units.__str__()+'|'+'مجموع تعداد واحد'+'|\n')
rdme_writer.close()


اصلاح فایل جداول برای github

In [305]:
# Read the content of the file
with open('../Tables.md', 'r') as file:
    content = file.read()

# Replace all occurrences of str1 with str2 in the content
new_content = content.replace('docs/', '')

# Write the modified content back to the file
with open('Tables.md', 'w') as file:
    file.write(new_content)

In [301]:
rdme_file_name = '../SUMMARY.md'
rdme_writer = open(rdme_file_name, 'w')
rdme_str = '\
* [صفحه اصلی](README.md)\n\
* [مشخصات کلی](general-specifications.md)\n\
* [جداول دروس](Tables.md)\n\
## دروس پایه\n\
'
rdme_writer.write(rdme_str)
cat_changed = False
r_path = 'base'
prev_cat = r_path
for index, row in df_sorted.iterrows():
    if 'پایه' in  row['c_cat']:
        r_path = 'base'
        curr_cat = r_path
    elif 'الزامی' in row['c_cat']:
        r_path = 'mandatory'
        curr_cat = r_path
        rdme_str = "دروس الزامی"
    elif 'اختیاری' in row['c_cat']:
        r_path = 'elective'
        curr_cat = r_path
        rdme_str = "دروس اختیاری"

    if curr_cat != prev_cat:
        rdme_writer.write('## ')
        rdme_writer.write(rdme_str)
        rdme_writer.write('\n')
    prev_cat = curr_cat
    # lesson_file_name = 'docs/'+r_path+'/'+row['en_file_name']+'.md'
    lesson_file_name = 'docs/'+row['en_file_name']
    rdme_str = '* ['+row['fa_title']+']('+lesson_file_name+')\n'
    rdme_writer.write(rdme_str)
rdme_writer.close()


فعلا فایلهای قبلی پوشه‌های دو گیت هاب به صورت دستی

In [302]:
!dir

 Volume in drive C has no label.
 Volume Serial Number is 823F-D2B7

 Directory of c:\temp\git\mkdocs-base\docs

04/30/2023  03:15 PM    <DIR>          .
04/30/2023  03:15 PM    <DIR>          ..
04/17/2023  04:09 AM               208 about.md
09/05/2023  05:16 PM    <DIR>          base
09/05/2023  05:16 PM    <DIR>          elective
05/15/2023  08:36 PM            21,740 index.md
09/05/2023  05:16 PM    <DIR>          mandatory
09/04/2023  04:32 PM           540,273 md2md.ipynb
09/05/2023  05:16 PM    <DIR>          output
04/26/2023  03:23 PM            13,180 Tables.md
04/26/2023  03:10 PM            13,900 Tables.md.bak
               5 File(s)        589,301 bytes
               6 Dir(s)  32,265,670,656 bytes free


حذف همه فایلهای قبلی

In [303]:
for git_name in ['curr/docs','cs-um.github.io/docs/docs']:
    folder_path = os.path.join('c:/temp/git/', git_name)    
    clear_doc_folders(folder_path)